In [12]:
from PIL import Image
import wave
from scipy.io import wavfile

# Audio Processing -----------------------------------------------------------------------------------------------------------------------------
# -> Outputs audio_bitstream
# audio_bitstream has the audio in binary and 16-bit depth
# Open the WAV file
wav_file = wave.open("./obiwan.wav", "r")

# Get the file properties
num_channels = wav_file.getnchannels()
sample_width = wav_file.getsampwidth()
sample_rate = wav_file.getframerate()
num_frames = wav_file.getnframes()
duration = num_frames / sample_rate  # Calculate the duration in seconds

print("Sample rate: ", sample_rate)

wav_file.close()

samplerate, data = wavfile.read('./obiwan.wav')

#convert audio to single channel
if num_channels != 1:
    data = data[:, 0]

#shift all data samples up by 2^15 to make all samples positive
data = data + 2**15

    #write single channel audio to new file
    #wavfile.write('./obiwan_single_channel.wav', samplerate, data)

#convert data to a bitstream 
audio_bitstream = ""
for sample in data:
    audio_bitstream += format(sample, '016b')

      
#print("Audio file properties:")
#print(f"Number of channels: {num_channels}")
#print(f"Sample width (bit depth): {sample_width * 8} bits")
#print(f"Sample rate: {sample_rate} Hz")
#print(f"Number of frames: {num_frames}")
#print(f"Duration: {duration:.2f} seconds")
#print(f"Bitstream length: {len(bitstream)} bits")
#print("-------------------------")




# Print audio bitstream
print("Audio Bit stream:")
print(audio_bitstream[:50])  
#print("Total number of audio bits:", len(bitstream))

# Save the encoded bit stream to a file
with open("audio_bitstream.txt", "w") as file:
    for i in range(0, len(audio_bitstream), 8):
        byte = audio_bitstream[i:i+8]  # Extracting 8 bits (1 byte)
        file.write(byte + "\n")

#Get length of audio_bitstream
audio_length = len(audio_bitstream)
audio_length_binary = format(audio_length, '024b')
print("length of audio", audio_length)
print("length of audio in binary", audio_length_binary)

# Attach audio length to the audio bitstream
audio_bitstream = audio_length_binary + audio_bitstream

#---------------------------------------------------------------------------------------------------------------------------------------
# Image processing ---------------------------------------------------------------------------------------------------------------------
# -> Outputs binary_pixels
# binary_pixels has the original image data in binary and in rgb format 

# Load the PNG image
image = Image.open("babyyoda.png")

# Get the size of the image
width, height = image.size

# Check size of image 
if width != 1000 or height != 1000:    # Dow we need this check?     
    print("Wrong image size!")
else:
    print("Image size is correct!\nStarting encryption...")

    # Convert the image to RGB mode 
    image_rgb = image.convert("RGB")

    # Get pixel data of image
    pixels = image_rgb.load()

    # Convert RGB values to binary and store in a 2D array
    binary_pixels = []
    for y in range(image.height):
        row = []
        for x in range(image.width):
            r, g, b = pixels[x, y]
            binary_r = bin(r)[2:].zfill(8)  # Convert red value to binary with 8 bits
            binary_g = bin(g)[2:].zfill(8)  # Convert green value to binary with 8 bits
            binary_b = bin(b)[2:].zfill(8)  # Convert blue value to binary with 8 bits
            row.append((binary_r, binary_g, binary_b))
        binary_pixels.append(row)

    # Access original pixel (1, 1) data
print("Original image pixel information in binary for pixel (1, 1)")
print(binary_pixels[0][0])
#----------------------------------------------------------------------------------------------------------------------------------------
# Image Encoding ------------------------------------------------------------------------------------------------------------------------
# -> outputs encoded_bitstream
# encoded_bitstream has the encoded image data in a binary bitstream 
# Every 24 bits of encoded_bitstream represents the information of 1 pixel
# Stored using big endian 

# Convert binary pixel data back to image and encode the image simultaneously
encoded_image = Image.new("RGB", (width, height))
encoded_pixels = encoded_image.load()

# Initialize variables to store the continuous bit stream
encoded_bitstream = "" # Will contain the encoded image bitstream for sending
stream_index = 0
encoded = [] #Contains the encoded image in a 2D array representing the RGB colours of the image 

# Iterate through each pixel in the image
for y in range(image.height):
    row = []
    for x in range(image.width):
        # Get the RGB values of the current pixel
        binary_r, binary_g, binary_b = binary_pixels[y][x]

        # Modify last two digits of each color component if audio stream has bits left
        if stream_index < len(audio_bitstream) - 1:
            r = binary_r[:-2] + audio_bitstream[stream_index] + audio_bitstream[stream_index + 1]
            stream_index += 2
        else:
            r = binary_r
            
        if stream_index < len(audio_bitstream) - 1:
            g = binary_g[:-2] + audio_bitstream[stream_index] + audio_bitstream[stream_index + 1]
            stream_index += 2
        else:
            g = binary_g
            
        if stream_index < len(audio_bitstream) - 1:
            b = binary_b[:-2] + audio_bitstream[stream_index] + audio_bitstream[stream_index + 1]
            stream_index += 2
        else:
            b = binary_b

        # Store modified RGB values in the encoded array
        row.append((r, g, b))

        # Append the modified bits to the continuous bit stream
        encoded_bitstream += r + g + b

        # Convert binary RGB values back to integer
        r_int = int(r, 2)
        g_int = int(g, 2)
        b_int = int(b, 2)
        # Set pixel value in the new image
        encoded_pixels[x, y] = (r_int, g_int, b_int)

    encoded.append(row)

# Output the first few bits of the continuous bit stream
print("First few bits of encoded image bit stream:")
print(encoded_bitstream[:24])

#print("first encoded bits: ", encoded[:1] )

# Save or display the encoded image
encoded_image.save("encoded_image.png")  

print("Done!")

# Saving to a txt file ------------------------------------------------------------------------------------------------------------------------
# Save the encoded bit stream to a file
with open("encoded_bitstream.txt", "w") as file:
    for i in range(0, len(encoded_bitstream), 8):
        byte = encoded_bitstream[i:i+8]  # Extracting 8 bits (1 byte)
        file.write(byte + "\n")



Sample rate:  44100
Audio Bit stream:
10000000000000001000000000000000100000000000000010
length of audio 3096576
length of audio in binary 001011110100000000000000
Image size is correct!
Starting encryption...
Original image pixel information in binary for pixel (1, 1)
('00000000', '00000000', '00000000')
First few bits of encoded image bit stream:
000000000000001000000011
Done!


In [13]:
from scipy.io import wavfile
import numpy as np

# Function to convert the bitstream to audio samples
def bits_to_audio(encoded_bitstream):
    audio_data = []
    # Convert the bit stream back to 16-bit signed integers
    for i in range(0, len(encoded_bitstream), 16):
        sample = encoded_bitstream[i:i+16]
        audio_data.append(int(sample, 2) - 2**15)
    return np.array(audio_data, dtype=np.int16)

# Function to decode the bit stream from the text file
def decode_bitstream_from_file(file_path):
    with open(file_path, "r") as file:
        encoded_bitstream = file.read()
    # Perform decoding here if needed
    # get the audio bitstream from the encoded image bitstream
    return encoded_bitstream

# Load the encoded bit stream from the text file and decode it
encoded_bitstream = decode_bitstream_from_file("encoded_bitstream.txt")

# Convert the decoded audio bitstream to audio data
decoded_audio_data = bits_to_audio(encoded_bitstream)

# Save the decoded audio data to a WAV file
output_file = "decoded_audio.wav"  # Specify the output file name
wavfile.write(output_file, 44100, decoded_audio_data)


ValueError: invalid literal for int() with base 2: '00000000\n0000001'